In [ ]:
import pandas as pd
import numpy as np

funding = pd.read_csv('/Users/duncanwan/Desktop/learning/Bitcoin/Clean_data/btcfunding_agg.csv',
                      usecols = ['calc_time', 'last_funding_rate'])

spot = pd.read_csv('/Users/duncanwan/Desktop/learning/Bitcoin/4hrs/spot/cleaned/BTCUSDT-4h-2025-04-08-cleaned.csv',
                   usecols = ['close_time','close','volume'])

prep = pd.read_csv('/Users/duncanwan/Desktop/learning/Bitcoin/4hrs/future/cleaned/BTCUSDT-4h-2025-04-08-cleaned.csv',
                   usecols = ['close_time','close','volume'])


spot["close_time"] = pd.to_datetime(spot["close_time"], utc=True).dt.ceil('4h')
prep["close_time"]  = pd.to_datetime(prep["close_time"],  utc=True).dt.ceil('4h')
funding["calc_time"]  = pd.to_datetime(funding["calc_time"],  utc=True)


spot = spot.rename(columns={"close_time":"bar_end",
                            "close":"spot_close",
                            "volume":"spot_vol"})
prep = prep.rename(columns={"close_time":"bar_end",
                           "close":"perp_close",
                           "volume":"perp_vol"})
funding= funding.rename(columns={"last_funding_rate":"funding"})

spot = spot.sort_values("bar_end").drop_duplicates("bar_end")
prep  = prep.sort_values("bar_end").drop_duplicates("bar_end")
funding = funding.sort_values("calc_time").drop_duplicates("calc_time")




In [ ]:
bars = (
    spot[['bar_end','spot_close','spot_vol']]
    .merge(prep[['bar_end','perp_close','perp_vol']], on='bar_end', how='inner')
    .sort_values('bar_end')
)



f8 = funding[['calc_time','funding']].sort_values('calc_time')

bars = pd.merge_asof(
    bars,                      # left: 4h bars
    f8,                        # right: 8h prints
    left_on='bar_end',
    right_on='calc_time',
    direction='backward',      # "duplicate until next update"
    allow_exact_matches=True
).drop(columns='calc_time').rename(columns={'funding':'fund'})

bars